<a href="https://colab.research.google.com/github/farhadhsn8/MLP/blob/master/implementationDynamicMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Attribute Information:

1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

## class:
1. Iris Setosa
2. Iris Versicolour
3. Iris Virginica

In [50]:
import matplotlib.pyplot as plt
from sklearn import datasets
import random
import numpy as np
import math
iris = datasets.load_iris()

In [51]:
iris = datasets.load_iris()
features = iris.data  
lables = iris.target
type(features)
features.shape   # (150, 4)
lables.shape    # (150,)
NUMBER_OF_OUTPUT = 2

In [200]:
class MLP:

  def __init__(self , feacturesOftrainingdata , lablesOftrainingdata):
    self.feacturesOftrainingdata = feacturesOftrainingdata
    self.lablesOftrainingdata = lablesOftrainingdata
    self.slidingHead = 0
    self.etha = float(input('enter learning rate :'))
    self.numberOfLayers = int(input('enter number of layers :'))
    self.layers =  np.empty(self.numberOfLayers,dtype=Layer)
    self.makeLayers()
    
    

  def makeLayers(self):
    for i in range(self.numberOfLayers):
      self.layers[i] = Layer(i , self)

  def train(self):
    return self.calculateMLP_outputForRow_k(self.getCurrentFeacherRow())


  def calculateMLP_outputForRow_k(self, X):
    return self.layers[self.numberOfLayers - 1].claculateLayerOutput()

  def getCurrentFeacherRow(self):
    return self.feacturesOftrainingdata[self.slidingHead]

  def getCurrentLableRow(self):
    return self.lablesOftrainingdata[self.slidingHead]



#--------------------------------------------------------------------------


class Layer:

  def __init__(self,layerAddress , MLP):
    self.MLP = MLP
    self.layerAddress = layerAddress
    self.numberOfPerceptrons = self.setNumberOfPerceptrons()
    self.activityFunction = ActivityFunction(self)
    self.perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron)
    self.perceptrons = self.makePerceptrons()
    self.output = np.full((self.numberOfPerceptrons), math.inf)


  def setNumberOfPerceptrons(self):
    if(self.layerAddress == 0 ):
      return  2 #features.shape[1] + 1  ###############
    if(self.layerAddress == self.MLP.numberOfLayers - 1 ):
      return  NUMBER_OF_OUTPUT
    return int(input('enter number of Perceptrons for  layer '+ str(self.layerAddress  ) + ' (start layer number from 0) : '))

  def makePerceptrons(self):
    perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron) 
    for i in range( self.numberOfPerceptrons ):
      perceptrons[i] = Perceptron( i , self)
    return perceptrons

  def getPreviosLayer(self):
    return self.layerAddress != 0 and self.MLP.layers[self.layerAddress - 1 ] or -1

  
  def getNextLayer(self):
    return self.layerAddress != self.MLP.numberOfLayers - 1 \
     and self.MLP.layers[self.layerAddress + 1 ] or -1


  def claculateLayerOutput(self):     # receive Vector   # return Vector
    if (self.output[0] !=   math.inf):
      print('oooookz',self.output)
      return self.output
    X = self.layerAddress==0 and self.MLP.getCurrentFeacherRow() or self.getPreviosLayer().claculateLayerOutput()
    output =  np.empty(self.numberOfPerceptrons)
    for i in range(self.numberOfPerceptrons):
      output[i] =(self.layerAddress == 0) and self.perceptrons[i].calculatePerceptronOutput([X[i]]) \
       or self.perceptrons[i].calculatePerceptronOutput(X)
    self.output = output
    return output

  def calculateDerivativeOfActivationFunction(self,net):
    return self.activityFunction.calculateDerivative(net)



#--------------------------------------------------------------------------





class ActivityFunction:
  
  def __init__(self,layer):
    self.layer = layer
    self.functionType = int(input('enter code of function for layer'+str(self.layer.layerAddress)+' =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :'))
  
  def runActivationFunction(self,x):
    if (self.functionType == 1) :
      return self.sigmoid(x)
    if (self.functionType == 2) :
      return self.tanh(x)
    if (self.functionType == 3) :
      return self.ReLU(x)
    if (self.functionType == 4) :
      return self.linear(x)

  def sigmoid(self, x):
    return 1 / (1 + math.exp(-x))

  def tanh(self , x):
    t=(math.exp(x)-math.exp(-x))/(math.exp(x)+math.exp(-x))
    return t

  def ReLU(self ,x):
    return max(0.0,x)

  def linear(self , x):
    return x

  def calculateDerivative(self , net):
    if (self.functionType == 1) :
      sig = self.sigmoid(net)
      return (1-sig)*sig
    if (self.functionType == 2) :
      return 1 - self.tanh(net)**2
    if (self.functionType == 3) :
      if(net<0):
        return 0
      return 1
    if (self.functionType == 4) :
      return 1



#--------------------------------------------------------------------------


class Perceptron:

  

  def __init__(self , perceptronNumber , layer ):   # [layerAddress  ,  perceptron] 
    self.perceptronNumber = perceptronNumber
    self.layer = layer
    self.numberOfInputs  =  self.getNumberOfInputs()
    self.inputBranchs =  np.empty(self.numberOfInputs,dtype=Layer)
    self.makeInputs()
    self.delta = math.inf

  def makeInputs(self):
    for i in range(self.numberOfInputs):
      self.inputBranchs[i] = InputBranch(self)

  def getNumberOfInputs(self):
    if(self.layer.layerAddress == 0 ):
      return  1
    return self.layer.getPreviosLayer().numberOfPerceptrons + 1

  def calculatePerceptronOutput(self , X):
        net = self.calculatePerceptronNet(X)
        return self.layer.activityFunction.runActivationFunction(net)

        
  def calculatePerceptronNet(self , X):    # X is input feature vector
        y=0
        # DONT FORGET BAIAS
        X = np.concatenate((X, [1]), axis=None)
        for i in range(self.numberOfInputs):
          y += self.inputBranchs[i].calculateBranchOutput(X[i])
        return y


  def getDelta(self):
    desiredOutput = self.layer.MLP.getCurrentLableRow()[self.perceptronNumber]
    X = (self.layer.layerAddress == 0)  and self.layer.MLP.getCurrentFeacherRow() or self.layer.getPreviosLayer().claculateLayerOutput()
    if(self.delta == math.inf):
      self.delta =  self.calculateDelta(X ,desiredOutput)
      return self.calculateDelta(X ,desiredOutput)
    print('ok',self.delta)
    return self.delta

  def calculateDelta(self,X , desiredOutput):  # X is input vector 
    net = self.calculatePerceptronNet(X)
    if(self.layer.layerAddress == self.layer.MLP.numberOfLayers - 1):     # perceptron in output layer
      return self.layer.calculateDerivativeOfActivationFunction(net) * ( desiredOutput - self.calculatePerceptronOutput(X))
    else:       # perceptron in hidden layer
      sigma = 0
      layerOutput = self.layer.claculateLayerOutput()
      for perceptron in self.layer.getNextLayer().perceptrons:
        sigma += perceptron.inputBranchs[self.perceptronNumber].w * perceptron.getDelta() 
      return self.layer.calculateDerivativeOfActivationFunction(net) * sigma


    


  
          


    







#--------------------------------------------------------------------------

class InputBranch:
  
  def __init__(self , perceptron):
    self.perceptron = perceptron
    self.w = self.setW()
    self.Wnew = self.w

  def setW(self):
    if(self.perceptron.layer.layerAddress == 0):
      return 1
    return 1#random.uniform(-3, 3)

  def calculateBranchOutput(self , x):
    return self.w * x 

  def updateW(self,desiredOutput):
    Wnew = self.perceptron.layer.MLP.etha * self.perceptron.getDelta(desiredOutput)





  



  

  










In [201]:
A = MLP([[1,2,3,]] , [[0,1,1]])

enter learning rate :1
enter number of layers :3
enter code of function for layer0 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
enter number of Perceptrons for  layer 1 (start layer number from 0) : 2
enter code of function for layer1 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
enter code of function for layer2 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4


In [202]:
# shape of MLP
for i in A.layers:
  print(i)
  for j in i.perceptrons:
    print( j.numberOfInputs)

1
1
3
3
3
3


In [203]:

A.train()




array([9., 9.])

In [204]:
for i in range(A.numberOfLayers):
  print('layer'+str(i))
  for j in range(A.layers[i].numberOfPerceptrons):
    print('percepron'+str(j))
    for k in range(A.layers[i].perceptrons[j].numberOfInputs):
      print(A.layers[i].perceptrons[j].inputBranchs[k].w)


layer0
percepron0
1
percepron1
1
layer1
percepron0
1
1
1
percepron1
1
1
1
layer2
percepron0
1
1
1
percepron1
1
1
1


In [205]:
A.layers[0].perceptrons[0].getDelta()


oooookz [1. 2.]
oooookz [1. 2.]
oooookz [4. 4.]
oooookz [4. 4.]
oooookz [4. 4.]
oooookz [4. 4.]
oooookz [4. 4.]
ok -9.0
oooookz [4. 4.]
ok -8.0
oooookz [1. 2.]
oooookz [4. 4.]
oooookz [4. 4.]
ok -9.0
oooookz [4. 4.]
ok -8.0
oooookz [4. 4.]
oooookz [4. 4.]
ok -9.0
oooookz [4. 4.]
ok -8.0
oooookz [1. 2.]
oooookz [1. 2.]
ok -17.0
oooookz [1. 2.]
ok -17.0


-34.0

In [123]:
x=9 
q = 1 / (1 + math.exp(-x)) 
q*(1-q)*(-q)

-0.00012336412542238054